In [1]:
%load_ext autoreload
%autoreload 2

from utils import load_config, set_environment_variables, num_tokens_from_messages, get_git_files, get_data_files
import sys
import os
config       = load_config('config.yaml')
set_environment_variables(config)
ACCESS_TOKEN = os.environ['GIT_ACCESS_TOKEN']
sys.path.append("/database")

In [2]:
from database import get_documentation_by_url, put_new_repository_documentation, get_file_documentation, delete_documentation_by_url, get_status, get_status_by_file, external_json_to_repo_overview_output
from repo_processor import process_file, parallel_process_files, download_and_process_repo_url, get_repo_overview
from OverviewChain import OverviewParser, ConfluenceOverviewChain
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.document_loaders import GithubFileLoader
from langchain.callbacks import get_openai_callback
from ConfluenceChain import ConfluenceChain, Parser
from pymongo import MongoClient
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
import pandas as pd
import nest_asyncio
import shutil
nest_asyncio.apply()


In [3]:
repo_url              = "https://github.com/Adarsh9616/Electricity_Billing_System/"
# repo_url              = "https://github.com/thuml/Time-Series-Library/"
supported_languages   = ['python', 'java', 'javascript']

# data_dict             = download_and_process_repo_url(repo_url, supported_languages)

In [31]:
# import boto3
# sqs       = boto3.client('sqs', 'us-east-1')
# queue_url = 'https://sqs.us-east-1.amazonaws.com/018192622412/doc-gen'
# response  = sqs.send_message(
#     QueueUrl=queue_url,
#     MessageBody=str("https://github.com/Adarsh9616/Electricity_Billing_System/")
# )

# """ or """

# https://bjxdbdicckttmzhrhnpl342k2q0pcthx.lambda-url.us-east-1.on.aws/?repo_url=https://github.com/Adarsh9616/Electricity_Billing_System/

In [30]:
# response = sqs.receive_message(
#     QueueUrl=queue_url,
#     MaxNumberOfMessages=1,
#     WaitTimeSeconds=1 # Long polling
# )

# messages = response.get('Messages', [])
# print(messages)
# while len(messages) != 0:
#     response = sqs.receive_message(
#         QueueUrl=queue_url,
#         MaxNumberOfMessages=1,
#         WaitTimeSeconds=1 # Long polling
#     )

#     messages = response.get('Messages', [])
#     print(messages)

[{'MessageId': '907db49d-4915-4b94-b643-bed34220873c', 'ReceiptHandle': 'AQEBPOEwO9zDxF4fb+4Av8xejsokvQWSp2x6ceBivpdwfZI9SReJic4X/GbJEpqBwYK+FmvRn1bYEEbCQwWdwnWjJrG3vxiQwyPNk0JqenT9mjMJm1YXqWQ+NeHvQ/GvK3O9PGpxisrC/TigpvIpntgImzFbA+u/R1puRfGf9kzgdLNalyJkOJanM8Dc46QfT37KB+Qch4J+BY97QklQmo//Xs9sWbfUivJxbarHCMi0isYsRa9L0Vjh2wTOYDIcBw1/tLDOySfWejeH7knNVUPdwsHMXpwtyK3mUm77jqJ2KBvqrfOHMJ73RKsjn0tn6mTc+LjFHPac4YKj2otWQmZDtTXER+PF+dKgey/RHojEKaKEnnm/LVkIn+PmstAbd2ER', 'MD5OfBody': '5e8c8de0e3a0c052ec15a8ed582f785c', 'Body': 'https://github.com/Adarsh9616/Electricity_Billing_System/'}]
[]


In [4]:
status_url = get_status(repo_url)
# get_status_by_file(repo_url, file_name)
status_url

{'repository_url': 'https://github.com/Adarsh9616/Electricity_Billing_System/',
 'overall_status': 'Completed',
 'file_level_status': {'src/splash.java': 'Completed',
  'src/generate.bill.java': 'Completed',
  'src/pay.bill.java': 'Completed',
  'src/conn.java': 'Completed',
  'src/customer.details.java': 'Completed',
  'src/login.java': 'Completed',
  'src/new.customer.java': 'Completed',
  'src/Project.java': 'Completed',
  'src/LastBill.java': 'Completed',
  'src/calculate.bill.java': 'Completed'}}

In [5]:
docs = get_documentation_by_url(repo_url)
docs.files

Now the file path is src/splash.java
Now the file path is src/generate.bill.java
Now the file path is src/pay.bill.java
Now the file path is src/conn.java
Now the file path is src/customer.details.java
Now the file path is src/login.java
Now the file path is src/new.customer.java
Now the file path is src/Project.java
Now the file path is src/LastBill.java
Now the file path is src/calculate.bill.java
Now the file path is src/splash.java


{'src/splash.java': FileConfluenceOutput(file_path='src/splash.java', overall_summary='The file defines a Java application that creates a splash screen for an Electricity Billing System. It uses a JFrame to display an animated splash screen before transitioning to a login screen.', packages={'java.awt': PackageDetail(usage='Used for creating and managing GUI components, such as layouts and images.', description='Provides classes for creating user interfaces and for painting graphics and images.'), 'javax.swing': PackageDetail(usage='Used for creating the GUI components, such as JFrame and JLabel.', description="Provides a set of 'lightweight' (all-Java language) components that, to the maximum degree possible, work the same on all platforms.")}, functions={'main': FunctionDetail(name='main', description='The entry point of the application. It creates and displays the splash screen with an animation effect.', class_declaration='', additional_details='The main function uses a for loop to

In [6]:
# delete_documentation_by_url(repo_url)

In [7]:
# name_of_repo = repo_url.split("/")[-2]
# res          = get_repo_overview(name_of_repo, str(docs.files))

In [4]:
repo_url            = "https://github.com/Adarsh9616/Electricity_Billing_System/"
url_to_process_repo = f"https://bjxdbdicckttmzhrhnpl342k2q0pcthx.lambda-url.us-east-1.on.aws/?repo_url={repo_url}"

import requests
response            = requests.get(url_to_process_repo)


In [5]:
response.json()

'Repository URL successfully enqueued: https://github.com/Adarsh9616/Electricity_Billing_System/'